In [1]:
import numpy as np
import pandas as pd

# 작업 2유형 


In [2]:
train = pd.read_csv("https://raw.githubusercontent.com/YoungjinBD/data/main/exam/10_2_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/YoungjinBD/data/main/exam/10_2_test.csv")
print(train.info())
print(test.info())

#테스트에서 결측치 있음을 확인함 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   biz_type        160 non-null    object 
 1   area            160 non-null    float64
 2   age             160 non-null    int64  
 3   num_households  160 non-null    int64  
 4   gas_totl        160 non-null    int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 6.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   biz_type        40 non-null     object 
 1   area            40 non-null     float64
 2   age             40 non-null     int64  
 3   num_households  40 non-null     int64  
 4   gas_totl        0 non-null      float64
dtypes: float64(2), int64(2), object(1)
memory usage: 1.7+ KB
None


In [3]:
#print(train.head())
print(train['gas_totl'].isna().sum())

0


In [4]:
train = train.loc[train['gas_totl']!=0,:]
#print(train)
#열방향으로 gas_totl를 drop한다 #drop 에서의 axis = 1과 계산할때의 axis =1은 다르더라 
train_x = train.drop(['gas_totl'],axis=1)
train_y = train['gas_totl']

test_x = test.drop(['gas_totl'],axis=1)
test_y = test['gas_totl']
print(train_x)
print(train_y)

        biz_type       area  age  num_households
0         public  93.291780   33              61
1    residential  97.509495   34             131
2    residential  97.140181    5              30
3    residential  65.789408   12             166
4          mixed  92.631000   28              14
..           ...        ...  ...             ...
155   industrial  89.174291   23              94
156       public  97.130755   14             145
157   industrial  76.334688   36             133
158  residential  62.510623   37             121
159       public  91.744768   31              26

[140 rows x 4 columns]
0       499
1      1156
2       392
3      1343
4       154
       ... 
155     851
156    1335
157    1157
158    1070
159     387
Name: gas_totl, Length: 140, dtype: int64


In [5]:
#모델의 성능하기 위해 훈련 데이터의 일부를 검증 데이터로 나누기 

from sklearn.model_selection import train_test_split

In [6]:
train_x,valid_x,train_y,valid_y = train_test_split(
    train_x,train_y,test_size=0.3,random_state=1
)

#분리 완료 / train 과 validation 데이터 나누기 완료 


In [7]:
#데이터 전처리 과정 실시
#object 형과 number 형으로 나눠서 따로 계산하는 것이 효율을 높이기 때문에 다음과 같이 나눠준다 

cat_columns = train_x.select_dtypes('object').columns
num_columns = train_x.select_dtypes('number').columns


In [8]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False,handle_unknown="ignore").set_output(transform="pandas")


In [9]:
train_x_cat_pre = ohe.fit_transform(train_x[cat_columns])
valid_x_cat_pre = ohe.fit_transform(valid_x[cat_columns])
test_x_cat_pre = ohe.fit_transform(test_x[cat_columns])

train_x_pre=pd.concat(
    [train_x_cat_pre,train_x[num_columns]],axis=1
)

valid_x_pre = pd.concat(
    [valid_x_cat_pre,valid_x[num_columns]],axis=1
)

test_x_pre = pd.concat(
    [test_x_cat_pre,test_x[num_columns]],axis=1
)


In [10]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=1)
rf.fit(train_x_pre,train_y)

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [13]:
from sklearn.metrics import root_mean_squared_error

pred_val = rf.predict(valid_x_pre)
print('valid rmse : ',root_mean_squared_error(valid_y,pred_val))


valid rmse :  75.47488439083246


In [19]:
test_pred = rf.predict(test_x_pre)
test_pred = pd.DataFrame(test_pred,columns=['pred'])
print(test_pred)

test_pred.to_csv('result.csv',index=False)

       pred
0    484.92
1   1038.76
2    352.11
3   1329.99
4    207.37
5    147.54
6   1134.22
7    160.70
8   1627.01
9   1306.51
10   167.51
11  1341.45
12   999.81
13  1515.01
14   987.53
15  1010.74
16  1127.17
17   198.22
18  1346.21
19   999.64
20   764.37
21   551.37
22  1132.73
23   176.80
24  1354.25
25  1343.84
26   139.43
27  1141.21
28  1140.33
29   167.01
30   173.05
31   799.94
32  1721.51
33  1364.62
34   335.08
35  1042.71
36  1320.13
37   277.12
38  1436.82
39   846.65
